In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import pandas as pd
import re
import pymorphy2
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import telegram
from collections import defaultdict
import random

In [2]:
TOKEN  = '1962265635:AAGl9sBcrGUNT9MXhXbxnHg0Q8vABEGExK8'

In [3]:
# Блок препоцессинга слов

def find_words(text, regex = re.compile('\w+')):

    if text:

        tokens =  regex.findall(text.lower())
        if tokens:
            return [w for w in tokens if w.isalpha() and len(w) >= 3]
    else:
        return []


# # Оставим в коллекции только существительные прилагательные

def lemmatize(words, lemmer = pymorphy2.MorphAnalyzer(), stopwords = stopwords.words('russian')):

    lemmas=[]

    if words:

        lemmas = [lemmer.parse(w)[0].normal_form.replace('ё','е') for w in words if w and
                      ('NOUN' in lemmer.parse(w)[0].tag or 'ADJF' in lemmer.parse(w)[0].tag)]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [4]:
# Блок отбора текста

def get_sim(word, text):
        
        s=[emb_mod.similarity(word,w) for w in text if w in emb_words]

        return max(s) if s else 0

def top_k_sim(req, text_prep,k=10):

    res=[]
    for text in text_prep:
        t=0
        for word in req :
            t+=get_sim(word, text)

        res.append(t)
    ind = np.argsort(res)[-k:][::-1] 
    return [data.text0.iloc[i] for i in ind if res[i]>0.5]

def top_k(req, text_prep,k=10):
    res=[]
    for text in text_prep:
        t=0
        for word in req :
            if word in text:
                t+=1
        res.append(t)
    ind = np.argsort(res)[-k:][::-1] 
    return [data.text0.iloc[i] for i in ind if res[i]>0.5]

In [5]:
# Блок подготовки данных

class Data_Prep():
    
    def __init__(self,text_file='aneks.sql', cat_file='cat.txt'):
        
        self.data=self.get_data(text_file)
        self.cat_list,self.cat_list_prep=self.get_cat_list(cat_file)
            
                    
    def get_cat_list(self, cat_file):
        
        with open(cat_file, encoding = 'utf-8') as f:
            cat=f.read()
   
        cat0=re.findall(r"(\d+) - ([^-]*)", cat)
        cat_list=[i[1][:-1] for i in cat0]
        cat_list_prep=[preprocess(i[1]) for i in cat0] 
        
        return cat_list,cat_list_prep
    
   
    def get_data(self, text_file):
        
        with open(text_file, encoding = 'utf-8') as f:
            text0=f.read()
        
        text0=re.findall(r"\d+, (\d+), '([^\']*)", text0)
        
        data=pd.DataFrame(text0,columns=['cat','text0'])
        
#         Для более быстрой работы алгоритма уменьшим объем данных

        ind_cut=[]
        ind_cut.extend(data[data.cat=='1'].index[:5000])       
        for i in data.cat.unique()[1:]:
            ind_cut.extend(data[data.cat==i].index[:100])
            
        data.text0=data.text0.apply(lambda x: x.replace('\\n','\n'))
        data.cat=data.cat.apply(int)
        data=data.iloc[ind_cut].reset_index(drop=True)
        data['text_prep']=data.text0.apply(preprocess)
        
#         Отберем 3000 тэгов для анализа косинусной близости, используя _idf

        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(data.text_prep.apply(lambda x:' '.join(x)).tolist())
        tags=list(dict(sorted(zip(vectorizer.get_feature_names(),vectorizer.idf_),
                              key=lambda item: item[1] )).keys())[:3000]
        data['tags']=data.text_prep.apply(lambda x: list(set([w for w in x if w in tags])))
        
        return data

In [6]:
data_prep=Data_Prep()
data=data_prep.data
cat_list=data_prep.cat_list
cat_list_prep=data_prep.cat_list_prep

In [7]:
# Предобученная модель FastText для получения эмбэддингов для расчета косинусной близости

emb_mod=KeyedVectors.load('213/model.model' )
emb_words=list(emb_mod.key_to_index.keys())

In [8]:
# Подбор анекдотов

class Get_Anek():
    
    def __init__(self,bot,chat_id,get_answer,text_file='aneks.sql', cat_file='cat.txt'):

            self.bot=bot
            self.get_answer=get_answer
            self.chat_id=chat_id
            self.greet_text=[]
            self.get_anek()
            
                
    def eval_answer(self, answer):
        
        if 'да' in answer or 'не' in answer:
            return 'да' in answer    
        else:
            return self.eval_answer(self.get_answer('непонятно, да или нет ?',self.chat_id))  
        
    
    def choose_anek(self,anek_list):
       
        for i in range(len(anek_list)):
            
            answer=self.get_answer(f'{anek_list[i]}\n Еще такой же ?',self.chat_id)

            if self.eval_answer(answer):

                if i == len(anek_list)-1: 

                    answer=self.get_answer('Все, таких больше не знаю, другой анекдот?\n ',self.chat_id)

                    return self.eval_answer(answer)

                continue

            else:
                answer=self.get_answer('Что-нибудь другое?\n',self.chat_id)
                return self.eval_answer(answer)
            
    
    def get_anek(self):
                      
        greet_text='Привет, я бот-анекдот и знаю много анекдотов на разные темы, например, такие:\n'
        self.bot.send_message(chat_id=self.chat_id, text=greet_text)
        self.bot.send_message(chat_id=self.chat_id, text=f'{cat_list}')
        
        while True:
                                   
            answer=self.get_answer('Напиши тему, или про что тебе рассказать анекдот\n', self.chat_id)
            
            answer_prep=preprocess(answer)
  
            if answer_prep in cat_list_prep:

                if self.anek_from_cat(answer_prep):
                
                    continue
                    
                else:
                    self.bot.send_message(chat_id=self.chat_id, text=random.choice(['Пока','Не скучай','Заходи еще']))
                    
                    break          
            else:
                
                if self.anek_from_prep(answer_prep):
                    
                    continue 
                    
                else:             

                    self.bot.send_message(chat_id=self.chat_id, text=random.choice(['Пока','Не скучай','Заходи еще']))

                    break
                    
#     выбор анекдота по категориям

    def anek_from_cat(self, answer_prep):
        
        anek_list_cat=data.text0[data.cat==cat_list_prep.index(answer_prep)+1].tolist()

        return   self.choose_anek(anek_list_cat)
    
#       по обработанному тексту

    def anek_from_prep(self,answer_prep):
        
        anek_list=top_k(answer_prep,data.text_prep)
              
        if not anek_list:
            
            answer=self.get_answer('Прямо такого не припоминаю, похожее может?',self.chat_id)
                            
            if self.eval_answer(answer) :
                    
                    return self.anek_from_tags(answer_prep)

        else:

            return self.choose_anek(anek_list)
                        
#     по тэгам на основе косинусной близости  

    def anek_from_tags(self,answer_prep):
        
        answer=self.bot.send_message(chat_id=self.chat_id, text='Мне понадобится время, чтобы вспомнить, подожди немного.')
       
        anek_list_sim=top_k_sim(answer_prep,data.tags)
               
        if not anek_list_sim:

            answer=self.get_answer('Извини, дружище, ничего смешного в голову не приходит\n Что-нибудь другое?',self.chat_id)

            return  self.eval_answer(answer)

        else:
            return self.choose_anek(anek_list_sim)


In [9]:
# Общение с ботом

class Conv_Bot():

    def __init__(self):
        
        self.bot = telegram.Bot(token=TOKEN)
        self.chats=defaultdict(object)
        self.start()
        
    def start(self):
        
        while True:
            
            try:          
                updates= self.bot.get_updates(timeout=1000)

                if updates:

                    if updates[-1].message.text=='/start': 
                        chat_id = updates[-1].message.chat_id
                        
                        if chat_id in self.chats:
#                             удаляем старый чат
                            self.chats.pop(chat_id)
                        self.chats[chat_id]=Get_Anek(self.bot, chat_id,self.get_answer)
              
            except:
                continue
    
    
    def get_answer(self,ask,chat_id):
           
        self.bot.send_message(chat_id=chat_id, text=ask)
    
        while True:

            try:            
                updates=self.bot.get_updates(timeout=10000)
       
                UPDATE_ID=updates[-1].update_id
                updates=self.bot.get_updates(offset=UPDATE_ID+1, timeout=10000)  
                
                if updates:

                    return updates[-1].message.text
            
            except:
                continue

In [ ]:
Conv_Bot()